In [1]:
import os
import shutil

import warnings
warnings.filterwarnings("ignore")

import nibabel as nib
import dicom2nifti
import numpy as np

from tqdm import tqdm
from scipy import ndimage
from matplotlib import pyplot as plt

from multiprocessing import Pool, cpu_count

import SimpleITK as sitk

In [2]:
LIDC_IDRI_NIFTI = './niigz'

# LIDC_IDRI_NIFTI = './LIDC-IDRI-NIFTI-RAI-LABEL-Union/' 

nifti_paths = sorted([ os.path.join(LIDC_IDRI_NIFTI, p) for p in os.listdir(LIDC_IDRI_NIFTI) if 'checkp' not in p ])
print( "Number of nifti files: %d files" % len(nifti_paths) )

Number of nifti files: 59 files


In [3]:
LIDC_IDRI_NIFTI_ZOOM = './niigz_075'
# LIDC_IDRI_NIFTI_ZOOM = './LIDC-IDRI-NIFTI-RAI-LABEL-Union-ZOOM/'
new_spacing = 0.75

if os.path.exists( LIDC_IDRI_NIFTI_ZOOM ):
    shutil.rmtree(LIDC_IDRI_NIFTI_ZOOM)

os.mkdir( LIDC_IDRI_NIFTI_ZOOM )

In [4]:
# If label: Please use order = 0 ,else order = 3

def zoom( data ):
    def do(path):
        load_nifti = nib.load(path)
        affine = load_nifti.affine
        images = load_nifti.get_fdata().astype(np.float64)
        header = load_nifti.header
        
        x_space, y_space, z_space = header['pixdim'][1:4]
        x_space, y_space, z_space
        
        
        zoom =  np.asarray( [ x_space, y_space, z_space ] ) / np.asarray( [new_spacing]*3 )
        
        zoomed_image = ndimage.zoom(images, zoom, order=3, mode='nearest', cval=0.0, prefilter=True)
        
#         round_image = np.around(zoomed_image!=0).astype(np.uint8)
        
        round_image = zoomed_image.astype(np.int16)
        
       
        new_affine = affine.copy()
        for i in range(3):
            new_affine[i, i] = new_affine[i, i] / zoom[i]
        
        new_images = nib.Nifti1Image(round_image,new_affine)
        dst_path = os.path.join( LIDC_IDRI_NIFTI_ZOOM, path.split( os.sep )[-1][:-7] + '-ZOOM.nii.gz' )
        new_images.to_filename(dst_path)
        
#         img = sitk.ReadImage(dst_path)
#         cc = sitk.ConnectedComponent(img)
#         sitk.WriteImage(cc, dst_path)
        
    try:
        do(data)
    except:
        with open('log.txt', 'a') as f:
            f.write("%s\n" % (data) )

In [5]:
for xc in nifti_paths:
    zoom(xc)

In [ ]:
zoom(nifti_paths[0])

In [8]:
with Pool(cpu_count()-2) as pool:
    r = list(tqdm(pool.imap(zoom, nifti_paths), total=len(nifti_paths)))

  0%|          | 0/58 [00:00&lt;?, ?it/s]

In [ ]:
exit()